**Назаров Дмитрий Алексеевич 214-322**

# Задание 4 Transfer Learning

## Задание

Решить вашу задачу классификации используя перенос обучения. **Transfer Learning** - это процесс дообучения на **новых данных** нейросети, которая была обучена до этого на других данных, например, ImageNet.

Определить задачу по классификации изображений основываясь на вашей теме диссертации. Если тема никак не связывается с изображениями, то придумать абстрактную задачу, например классификация радиодеталей.

В рамках задачи выбрать количество классов объектов. На каждый класс подобрать минимум по 30 изображений и распределить их на обучающую (train) и проверочную выборку (val).

Выбрать 2-3 уже обученные модели, дообучить их на ваших данных и оценить результат. Обученную модель используйте для извлечения признаков, то есть нужно "заморозить" все веса, кроме последних полносвязных слоев. Их мы создаем заново и в соответствии с вашим количеством классов.

## Источники

1. [Transfer learning for computer vision tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [2]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torchvision.io import read_image
from torchvision.models import ResNet101_Weights, ResNet18_Weights
import copy

C:\Users\Dmitr\AppData\Local\Temp/ipykernel_8200/2708429422.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Преобразование обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir =  "./hymenoptera_data" # папка с данными. Если запускаете в Colab, нужно скопировать данные к себе в директорию и примонтировать диск. Если запускаете локально --- скачайте данные.

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# специальный класс для загрузки данных в виде батчей
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

## Извлечение признаков

[Список](https://pytorch.org/vision/stable/models.html#classification) предобученных моделей в Pytorch.

### Загрузка модели

In [4]:
num_classes = 2 # Количество классов в вашей задаче

weights = ResNet18_Weights.DEFAULT
model_conv = torchvision.models.resnet18(weights=weights) # Выбирите 2-3 модели из списка и дообучите их (pretrained=True)
# weights = ResNet101_Weights.DEFAULT
# model_conv = torchvision.models.resnet101(weights=weights)
for param in model_conv.parameters():
    param.requires_grad = False

# Параметры вновь созданных моделей по умолчанию имеют requires_grad=True
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Обратите внимание, что оптимизируются только параметры последнего слоя, в отличие от предыдущего.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Уменьшение LR в 0,1 раза каждые 7 эпох
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Dmitr/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

### Функция для обучения

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Обучение и оценка

In [6]:
model_ft = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=12)

Epoch 0/11
----------
train Loss: 0.8541 Acc: 0.3750
val Loss: 0.5915 Acc: 0.6875

Epoch 1/11
----------
train Loss: 0.4690 Acc: 0.8125
val Loss: 0.2276 Acc: 1.0000

Epoch 2/11
----------
train Loss: 0.4355 Acc: 0.8333
val Loss: 0.1295 Acc: 1.0000

Epoch 3/11
----------
train Loss: 0.4255 Acc: 0.7708
val Loss: 0.1253 Acc: 1.0000

Epoch 4/11
----------
train Loss: 0.3310 Acc: 0.8750
val Loss: 0.1425 Acc: 0.9375

Epoch 5/11
----------
train Loss: 0.3626 Acc: 0.8750
val Loss: 0.1644 Acc: 0.9062

Epoch 6/11
----------
train Loss: 0.4132 Acc: 0.8333
val Loss: 0.0915 Acc: 0.9688

Epoch 7/11
----------
train Loss: 0.3370 Acc: 0.8333
val Loss: 0.0911 Acc: 0.9688

Epoch 8/11
----------
train Loss: 0.2240 Acc: 0.8958
val Loss: 0.0604 Acc: 1.0000

Epoch 9/11
----------
train Loss: 0.1413 Acc: 1.0000
val Loss: 0.0608 Acc: 1.0000

Epoch 10/11
----------
train Loss: 0.3025 Acc: 0.8958
val Loss: 0.0522 Acc: 1.0000

Epoch 11/11
----------
train Loss: 0.2100 Acc: 0.9375
val Loss: 0.0629 Acc: 1.0000

Tr

## Анализ данных

Проанализировать данные и сделать выводы о возможных проблемах.

Изображения для дообучения: "болты" и "гайки"
1) Модель "resnet101": обучение 12 эпох, точность 0.843750
2) Модель "resnet18": обучение 25 эпох, точность 1

Модель с более меньшим кол-вом слоёв выполнила свою работу точнее. Такое поведение объяснятеся тем, что веса для этой модели были подобраны более подходящие.